In [1]:
import MySQLdb

myvars = {}
with open("ksunchu-password") as myfile:
 for line in myfile:
    name, var = line.partition("=")[::2]
    myvars[name.strip()] = var.strip()

conn = MySQLdb.connect(host="127.0.0.1", user=myvars['username'], passwd=myvars['password'], db='omop')

omopcursor = conn.cursor()
print(conn)

<_mysql.connection open to '127.0.0.1' at 0x558042730010>


In [2]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from dask.distributed import Client

In [3]:
df_PAYER_PLAN_PERIOD = pd.read_sql('select * from `PAYER_PLAN_PERIOD`', con=conn)

/tmp/ipykernel_117697/2801689743.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_PAYER_PLAN_PERIOD = pd.read_sql('select * from `PAYER_PLAN_PERIOD`', con=conn)


In [4]:
print('Length - PAYER_PLAN_PERIOD: ', len(df_PAYER_PLAN_PERIOD))

Length - PAYER_PLAN_PERIOD:  0


In [5]:
print(df_PAYER_PLAN_PERIOD.head())

Empty DataFrame
Columns: [payer_plan_period_id, person_id, payer_plan_period_start_date, payer_plan_period_end_date, payer_concept_id, payer_source_value, payer_source_concept_id, plan_concept_id, plan_source_value, plan_source_concept_id, sponsor_concept_id, sponsor_source_value, sponsor_source_concept_id, family_source_value, stop_reason_concept_id, stop_reason_source_value, stop_reason_source_concept_id]
Index: []


In [6]:
import MySQLdb

myvars = {}
with open("ksunchu-password") as myfile:
 for line in myfile:
    name, var = line.partition("=")[::2]
    myvars[name.strip()] = var.strip()

mimicivconn = MySQLdb.connect(host="127.0.0.1", user=myvars['username'], passwd=myvars['password'], db='mimiciv')

mimicivcursor = conn.cursor()
print(conn)

<_mysql.connection open to '127.0.0.1' at 0x558042730010>


In [7]:
df_admissions_mimiciv = pd.read_sql('select * from admissions', con=mimicivconn)
print(df_admissions_mimiciv.head()) 

/tmp/ipykernel_117697/3412243236.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_admissions_mimiciv = pd.read_sql('select * from admissions', con=mimicivconn)


   subject_id   hadm_id           admittime           dischtime deathtime  \
0    10467237  20000019 2159-03-20 21:08:00 2159-03-23 16:54:00       NaT   
1    16925328  20000024 2151-05-25 21:44:00 2151-05-26 18:36:00       NaT   
2    19430048  20000034 2174-05-22 19:13:00 2174-05-24 17:27:00       NaT   
3    18910522  20000041 2143-09-03 07:15:00 2143-09-06 13:50:00       NaT   
4    11146739  20000057 2190-01-15 17:07:00 2190-01-18 16:05:00       NaT   

                admission_type admit_provider_id  admission_location  \
0                     EW EMER.            P25U4M      EMERGENCY ROOM   
1                     EW EMER.            P54Q2B      EMERGENCY ROOM   
2           DIRECT OBSERVATION            P30SZF  PHYSICIAN REFERRAL   
3  SURGICAL SAME DAY ADMISSION            P48VXK  PHYSICIAN REFERRAL   
4               EU OBSERVATION            P77SO2      EMERGENCY ROOM   

         discharge_location insurance language marital_status  \
0                      HOME     Other  

In [8]:
ddf_admissions_mimiciv = dd.from_pandas(df_admissions_mimiciv).compute()

In [9]:
df_admissions_mimiciv = ''

In [10]:
print(ddf_admissions_mimiciv)

        subject_id   hadm_id           admittime           dischtime  \
0         10467237  20000019 2159-03-20 21:08:00 2159-03-23 16:54:00   
1         16925328  20000024 2151-05-25 21:44:00 2151-05-26 18:36:00   
2         19430048  20000034 2174-05-22 19:13:00 2174-05-24 17:27:00   
3         18910522  20000041 2143-09-03 07:15:00 2143-09-06 13:50:00   
4         11146739  20000057 2190-01-15 17:07:00 2190-01-18 16:05:00   
...            ...       ...                 ...                 ...   
431226    10382924  29999723 2170-08-09 18:56:00 2170-08-10 18:30:00   
431227    11326722  29999745 2160-07-05 22:24:00 2160-07-12 12:50:00   
431228    12133002  29999809 2172-09-24 21:27:00 2172-09-30 17:04:00   
431229    16572164  29999828 2180-10-27 07:15:00 2180-10-30 17:25:00   
431230    16549335  29999928 2165-07-28 17:22:00 2165-07-31 11:53:00   

       deathtime               admission_type admit_provider_id  \
0            NaT                     EW EMER.            P25U4M   
1

In [11]:
import uuid
import random

def generate_numeric_uuid():
    return random.randint(0, 9223372036854775807)
ids = [generate_numeric_uuid() for _ in range(len(ddf_admissions_mimiciv))]
ddf_admissions_mimiciv['payer_plan_period_id'] = ids
print(ddf_admissions_mimiciv['payer_plan_period_id'])

0         9180735397357200780
1         2759781148974022072
2         1330858448374658694
3         8430479586378334773
4         4082152302333149377
                 ...         
431226    3569940465342147155
431227    2567939524791348279
431228    8382757812386971702
431229    7801141846469034466
431230     685350496609261341
Name: payer_plan_period_id, Length: 431231, dtype: int64


In [12]:
ddf_admissions_mimiciv['payer_plan_period_start_date'] = np.nan
ddf_admissions_mimiciv['payer_plan_period_end_date'] = np.nan
ddf_admissions_mimiciv['payer_source_concept_id'] = np.nan
ddf_admissions_mimiciv['plan_concept_id'] = np.nan
ddf_admissions_mimiciv['plan_source_value'] = np.nan
ddf_admissions_mimiciv['plan_source_concept_id'] = np.nan
ddf_admissions_mimiciv['sponsor_concept_id'] = np.nan
ddf_admissions_mimiciv['sponsor_source_value'] = np.nan
ddf_admissions_mimiciv['sponsor_source_concept_id'] = np.nan
ddf_admissions_mimiciv['family_source_value'] = np.nan
ddf_admissions_mimiciv['stop_reason_concept_id'] = np.nan
ddf_admissions_mimiciv['stop_reason_source_value'] = np.nan
ddf_admissions_mimiciv['stop_reason_source_concept_id'] = np.nan
print(ddf_admissions_mimiciv)

        subject_id   hadm_id           admittime           dischtime  \
0         10467237  20000019 2159-03-20 21:08:00 2159-03-23 16:54:00   
1         16925328  20000024 2151-05-25 21:44:00 2151-05-26 18:36:00   
2         19430048  20000034 2174-05-22 19:13:00 2174-05-24 17:27:00   
3         18910522  20000041 2143-09-03 07:15:00 2143-09-06 13:50:00   
4         11146739  20000057 2190-01-15 17:07:00 2190-01-18 16:05:00   
...            ...       ...                 ...                 ...   
431226    10382924  29999723 2170-08-09 18:56:00 2170-08-10 18:30:00   
431227    11326722  29999745 2160-07-05 22:24:00 2160-07-12 12:50:00   
431228    12133002  29999809 2172-09-24 21:27:00 2172-09-30 17:04:00   
431229    16572164  29999828 2180-10-27 07:15:00 2180-10-30 17:25:00   
431230    16549335  29999928 2165-07-28 17:22:00 2165-07-31 11:53:00   

       deathtime               admission_type admit_provider_id  \
0            NaT                     EW EMER.            P25U4M   
1

In [13]:
admission_insurance_mapping = {
    "Other": None,
    "Medicare": 280,
    "Medicaid": 289,
}

ddf_admissions_mimiciv['payer_concept_id'] = ddf_admissions_mimiciv['insurance'].map(admission_insurance_mapping)
ddf_admissions_mimiciv['payer_concept_id'] = ddf_admissions_mimiciv['payer_concept_id'].astype('Int64')

print(ddf_admissions_mimiciv[['insurance', 'payer_concept_id']])

       insurance  payer_concept_id
0          Other              <NA>
1       Medicare               280
2       Medicare               280
3          Other              <NA>
4       Medicare               280
...          ...               ...
431226     Other              <NA>
431227     Other              <NA>
431228  Medicare               280
431229     Other              <NA>
431230     Other              <NA>

[431231 rows x 2 columns]


In [14]:
ddf_admissions_mimiciv_copy =  ddf_admissions_mimiciv.copy()
ddf_admissions_mimiciv_copy

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,...,plan_source_value,plan_source_concept_id,sponsor_concept_id,sponsor_source_value,sponsor_source_concept_id,family_source_value,stop_reason_concept_id,stop_reason_source_value,stop_reason_source_concept_id,payer_concept_id
0,10467237,20000019,2159-03-20 21:08:00,2159-03-23 16:54:00,NaT,EW EMER.,P25U4M,EMERGENCY ROOM,HOME,Other,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
1,16925328,20000024,2151-05-25 21:44:00,2151-05-26 18:36:00,NaT,EW EMER.,P54Q2B,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,280
2,19430048,20000034,2174-05-22 19:13:00,2174-05-24 17:27:00,NaT,DIRECT OBSERVATION,P30SZF,PHYSICIAN REFERRAL,<NA>,Medicare,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,280
3,18910522,20000041,2143-09-03 07:15:00,2143-09-06 13:50:00,NaT,SURGICAL SAME DAY ADMISSION,P48VXK,PHYSICIAN REFERRAL,SKILLED NURSING FACILITY,Other,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
4,11146739,20000057,2190-01-15 17:07:00,2190-01-18 16:05:00,NaT,EU OBSERVATION,P77SO2,EMERGENCY ROOM,<NA>,Medicare,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431226,10382924,29999723,2170-08-09 18:56:00,2170-08-10 18:30:00,NaT,EW EMER.,P886Y0,EMERGENCY ROOM,HOME,Other,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
431227,11326722,29999745,2160-07-05 22:24:00,2160-07-12 12:50:00,NaT,EW EMER.,P64BMC,EMERGENCY ROOM,HOME,Other,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
431228,12133002,29999809,2172-09-24 21:27:00,2172-09-30 17:04:00,NaT,EW EMER.,P14D72,EMERGENCY ROOM,HOME,Medicare,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,280
431229,16572164,29999828,2180-10-27 07:15:00,2180-10-30 17:25:00,NaT,SURGICAL SAME DAY ADMISSION,P843YV,PHYSICIAN REFERRAL,HOME,Other,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>


In [15]:
payer_plan_period_data = ddf_admissions_mimiciv_copy[['payer_plan_period_id', 'subject_id', 'payer_plan_period_start_date', 'payer_plan_period_end_date', 'payer_concept_id', 'insurance', 'payer_source_concept_id', 'plan_concept_id', 'plan_source_value', 'plan_source_concept_id', 'sponsor_concept_id', 'sponsor_source_value', 'sponsor_source_concept_id', 'family_source_value', 'stop_reason_concept_id', 'stop_reason_source_value', 'stop_reason_source_concept_id']]

for index, row in payer_plan_period_data.iterrows():
    row = row.where(pd.notnull(row), None)
    
    sql = "INSERT INTO PAYER_PLAN_PERIOD (payer_plan_period_id, person_id, payer_plan_period_start_date, payer_plan_period_end_date, payer_concept_id, payer_source_value, payer_source_concept_id, plan_concept_id, plan_source_value, plan_source_concept_id, sponsor_concept_id, sponsor_source_value, sponsor_source_concept_id, family_source_value, stop_reason_concept_id, stop_reason_source_value, stop_reason_source_concept_id) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    
    omopcursor.execute(sql, (
        row['payer_plan_period_id'],
        row['subject_id'],
        row['payer_plan_period_start_date'],
        row['payer_plan_period_end_date'],
        row['payer_concept_id'],
        row['insurance'],
        row['payer_source_concept_id'],
        row['plan_concept_id'],
        row['plan_source_value'],
        row['plan_source_concept_id'],
        row['sponsor_concept_id'],
        row['sponsor_source_value'],
        row['sponsor_source_concept_id'],
        row['family_source_value'],
        row['stop_reason_concept_id'],
        row['stop_reason_source_value'],
        row['stop_reason_source_concept_id']
    ))

conn.commit()
print("Insertion was successful.")

Insertion was successful.
